In [2]:
import dotenv
import os
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("API_KEY", "")
os.environ["OPENAI_BASE_URL"] = os.getenv("BASE_URL", "")


## 文档解析与切分

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import pickle
from uuid import uuid4

def split_docs(documents, filepath, chunk_size=500, chunk_overlap=50, seperators=['\n\n\n', '\n\n'], force_split=False):
    if os.path.exists(filepath) and not force_split:
        return pickle.load(open(filepath, 'rb'))
    
    splitter = RecursiveCharacterTextSplitter(
        separators=seperators,
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    split_docs = splitter.split_documents(documents)
    for doc in split_docs:
        doc.metadata['uuid'] = str(uuid4())

    pickle.dump(split_docs, open(filepath, 'wb'))
    return split_docs

loader = PyPDFLoader("data/2024全球经济金融展望报告.pdf")
documents = loader.load()

pattern = r"^全球经济金融展望报告\n中国银行研究院 \d+ 2024 年"
merged_docs = [Document(page_content='\n'.join(re.sub(pattern, "", doc.page_content) for doc in documents))]
splitted_docs = split_docs(merged_docs, os.path.join("outputs", 'split_docs.pkl'), chunk_size=500, chunk_overlap=50)
uuid2doc = {doc.metadata['uuid']: doc for doc in splitted_docs}

## QA抽取

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from typing import List, Optional
from pydantic import BaseModel, Field

class QAItem(BaseModel):
    """
    定义单个问答（QA）项的结构。
    """
    question: str = Field(..., description="用户提出的问题。")
    context: Optional[str] = Field(None, description="用于生成答案的上下文信息，如果没有则为 None。")
    answer: str = Field(..., description="对问题的回答。")

class QAResponse(BaseModel):
    """
    定义最终的 JSON 响应格式，它是一个 QAItem 对象的列表。
    """
    results: List[QAItem] = Field(..., description="一个包含所有问答结果的列表。")

gen_parser = PydanticOutputParser(pydantic_object=QAResponse)

gen_template = PromptTemplate.from_template(
    template=
    """我会给你一段文本（<document></document>之间的部分），你需要阅读这段文本，分别针对这段文本生成8个问题、用户回答这个问题的上下文，和基于上下文对问题的回答。
    对问题、上下文、答案的要求：
    问题要与这段文本相关，不要询问类似“这个问题的答案在哪一章”这样的问题
    上下文：上下文必须与原始文本的内容保持一致，不要进行缩写、扩写、改写、摘要、替换词语等
    答案：回答请保持完整且简洁，无须重复问题。答案要能够独立回答问题，而不是引用现有的章节、页码等

    返回结果以JSON形式组织，格式为{format}。
    如果当前文本主要是目录，或者是一些人名、地址、电子邮箱等没有办法生成有意义的问题时，可以返回[]。

    下方是文本：
    <document>
    {document}
    </document>

    请生成结果：
    """,
    partial_variables={"format": gen_parser.get_format_instructions()}
)
print(gen_template)

input_variables=['document'] input_types={} partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"QAItem": {"description": "定义单个问答（QA）项的结构。", "properties": {"question": {"description": "用户提出的问题。", "title": "Question", "type": "string"}, "context": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "用于生成答案的上下文信息，如果没有则为 None。", "title": "Context"}, "answer": {"description": "对问题的回答。", "title": "Answer", "type": "string"}}, "required": ["question", "answer"], "title": "QAItem", "type": "object"}}, "description": "定义最终的 J

In [ ]:
from langchain.chat_models import init_chat_model
llm = init_chat_model(
    model="openai:gpt-4o-mini",
    top_p=0.95,
    temperature=0.85
)
chain = gen_template | llm | gen_parser

In [15]:
detailed_qa_dict = chain.batch([{"document": doc.page_content} for doc in splitted_docs])

In [16]:
detailed_qa_dict[1].results

[QAItem(question='2023年全球经济增长的动力是什么?', context='2023 年，全球经济增长动力持续回落。分区域看，各国复苏存在较大差异，发达经济体增速明显放缓，新兴经济体增速与2022 年大致持平。', answer='2023年全球经济增长的动力主要受到地缘政治冲突、高通胀和货币政策紧缩等因素的影响。'),
 QAItem(question='2023年全球经济预计的GDP增速是多少?', context='预计2023年全球GDP 增速为2.7%（市场汇率法），较2022 年下降0.3 个百分点。', answer='2023年全球经济预计的GDP增速为2.7%。'),
 QAItem(question='全球供应链在2023年有何变化?', context='生产端，全球供应链持续恢复，但生产景气度逐渐回落。', answer='在2023年，全球供应链持续恢复，但生产景气度逐渐回落。'),
 QAItem(question='欧美央行的货币政策在2023年有什么趋势?', context='欧美央行货币政策延续收紧态势，但步伐整体放缓。', answer='在2023年，欧美央行的货币政策延续收紧态势，但整体步伐有所放缓。'),
 QAItem(question='高利率环境对债券融资需求有何影响?', context='高利率环境抑制债券融资需求，债券违约风险持续上升。', answer='高利率环境抑制了债券融资需求，并持续增加了债券违约风险。'),
 QAItem(question='预计2024年全球经济复苏的状况如何?', context='展望2024 年，预计全球经济复苏将依旧疲软，主要经济体增长态势和货币政策将进一步分化。', answer='预计2024年全球经济复苏仍将疲软，主要经济体的增长态势和货币政策将进一步分化。'),
 QAItem(question='日本央行在未来的货币政策可能会如何变化?', context='日本央行可能退出负利率政策，跨境资本回流美国趋势将放缓。', answer='未来，日本央行可能会退出负利率政策。'),
 QAItem(question='本期报告分析了哪些专题?', context='本期报告分别对海湾六国经济发展与投资前景、高利率和高债务对美国房地产市场脆弱性的影

In [28]:
import json
with open("outputs/qa.json", "wt") as f:
    for i, response in enumerate(detailed_qa_dict):
        id = splitted_docs[i].metadata["uuid"]
        for qaitem in response.results:
            qadict = qaitem.model_dump()
            qadict["uuid"] = id
            f.write(json.dumps(qadict, ensure_ascii=False) + '\n')